In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import pandas as pd

from tqdm import tqdm

In [15]:
df = pd.read_csv("data\df_01.csv", index_col=[0])

In [16]:
lithology_keys = {30000: 0,
                  65030: 1,
                  65000: 2,
                  80000: 3,
                  74000: 4,
                  70000: 5,
                  70032: 6,
                  88000: 7,
                  86000: 8,
                  99000: 9,
                  90000: 10,
                  93000: 11}

In [17]:
df['FORCE_2020_LITHOFACIES_LITHOLOGY'] = df['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_keys)

In [18]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
class CustomDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame, feature_names: list, target_name: str):
        self.X = torch.tensor(dataframe[feature_names].values, dtype=torch.float32)
        self.X = self.X.reshape(self.X.shape[0], 1, self.X.shape[1])
        self.y = torch.tensor(dataframe[target_name].values, dtype=torch.float32)
        self.y = self.y.type(torch.LongTensor)
        

    def __len__(self) -> int:
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [20]:
feature_names = ['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RMED', 'RDEP', 'RHOB',
                 'GR', 'NPHI', 'PEF', 'DTC', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'ROPA']

target_name = 'FORCE_2020_LITHOFACIES_LITHOLOGY'

train_dataset = CustomDataset(dataframe=df, feature_names=feature_names, target_name=target_name)
train_data = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, )

In [21]:
len(feature_names)

18

In [22]:
model = nn.Sequential(
    nn.Conv1d(1, 64, 3, padding=1),
    nn.ReLU(),
    nn.Conv1d(64, 64, 3, padding=1),
    nn.ReLU(),
    nn.Conv1d(64, 64, 3, padding=1),
    nn.ReLU(),


    # nn.Conv1d(64, 128, 3, padding=1),
    # nn.ReLU(),
    # nn.Conv1d(128, 128, 3, padding=1),
    # nn.ReLU(),
    # nn.Conv1d(128, 128, 1, padding=1),
    nn.Flatten(),
    nn.Linear(18, 32), # 1152
    nn.ReLU(),
    nn.Linear(32, 12),
    nn.Softmax()
)

In [23]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.001)
loss_function = nn.CrossEntropyLoss()

In [24]:
epochs = 5
model.train()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=18, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=12, bias=True)
  (4): Softmax(dim=None)
)

In [25]:
for e in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean
        train_tqdm.set_description(f'Epoch [{e+1}/{epochs}], loss_mean={loss_mean:.3f} ')

  0%|          | 0/1898 [00:00<?, ?it/s]

c:\Users\tatya\Documents\well_log\venv\lib\site-packages\torch\nn\modules\module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


torch.Size([64, 1, 18])
torch.Size([64])


Epoch [1/5], loss_mean=2.550 :   0%|          | 3/1898 [00:00<04:08,  7.64it/s]

torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])


...

torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])


Epoch [5/5], loss_mean=2.532 :  99%|█████████▉| 1878/1898 [00:38<00:00, 75.18it/s]

torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])


Epoch [5/5], loss_mean=2.532 : 100%|██████████| 1898/1898 [00:39<00:00, 48.63it/s]

torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([64, 1, 18])
torch.Size([64])
torch.Size([16, 1, 18])
torch.Size([16])


In [26]:
class ConvNet(nn.Module):
    def __init__(self, n_class, in_channels):
        super().__init__()

        self.conv_1 = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=2, padding=1)
        self.conv_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, padding=1)
